# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import json
import re

import sklearn
import numpy as np
import nltk
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files

from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

moviedir = r'./dataset_movies/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [4]:
#!pip3 install --upgrade watson-machine-learning-client 

In [5]:
#!pip3 install --upgrade watson-developer-cloud

In [6]:
#!pip3 install --upgrade wget

In [7]:
#!conda install nltk

In [8]:
#!pip3 show scikit-learn

In [9]:
#!pip3 uninstall scikit-learn

In [10]:
#!pip3 install scikit-learn==0.20.3

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [11]:
from watson_developer_cloud import NaturalLanguageUnderstandingV1

from watson_machine_learning_client import WatsonMachineLearningAPIClient

NatLearUnder = {
  "apikey": "RqzBP7gaBbeFX2v4gAlUZTw2eBB4w1H9quVVIHl14uRC",
  "iam_apikey_description": "Auto-generated for key 8655e708-3ee9-4138-a0e7-82a092cc9a9e",
  "iam_apikey_name": "Auto-generated service credentials",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/62c6cea1393d4576b56fdd7641e7d720::serviceid:ServiceId-50fce8b9-44bb-4fce-a4a6-aba1425edc1a",
  "url": "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/cf5cd667-3142-4d27-81fc-39e37a150e2a"
}

Watsonml = {
  "apikey": "w2NAeiIZQo4rvz6PUsAklPaweCEqiSPj2cOKbu9PAKIK",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/62c6cea1393d4576b56fdd7641e7d720:c081364d-d603-4888-9337-272d3117a955::",
  "iam_apikey_name": "auto-generated-apikey-0457564c-a895-4da7-95e1-8c955d0b4df6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/62c6cea1393d4576b56fdd7641e7d720::serviceid:ServiceId-50fce8b9-44bb-4fce-a4a6-aba1425edc1a",
  "instance_id": "c081364d-d603-4888-9337-272d3117a955",
  "url": "https://us-south.ml.cloud.ibm.com"
}



#NaturalLUnder = NaturalLanguageUnderstandingV1(
#    version='2018-11-16',
#    iam_api_key='RqzBP7gaBbeFX2v4gAlUZTw2eBB4w1H9quVVIHl14uRC',
#    url='https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/cf5cd667-3142-4d27-81fc-39e37a150e2a')

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [12]:
client = WatsonMachineLearningAPIClient(Watsonml)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [13]:
model_properties = { client.repository.ModelMetaNames.AUTHOR_NAME       : "Ricardo Naranjo", 
                     client.repository.ModelMetaNames.NAME              : "CritiKon",
                     client.repository.ModelMetaNames.FRAMEWORK_NAME    : "scikit-learn",
                     client.repository.ModelMetaNames.FRAMEWORK_VERSION : "0.19",
                     client.repository.ModelMetaNames.RUNTIME_NAME      : 'python',
                     client.repository.ModelMetaNames.RUNTIME_VERSION   : '3.6'
                   }

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [14]:
def preprocesar(lista, lista_lista, stopers, decodeit):
    for i in lista:
        if (decodeit):
            letters = i.decode("utf-8")
            letters_only = re.sub("\n", "", letters)
        else: 
            letters_only = re.sub("\n", "", i)
        tokenizer = RegexpTokenizer("[\w]+")
        letteres = tokenizer.tokenize(letters_only)
        leters = " ".join(letteres)
        word_tokens = nltk.word_tokenize(leters)
        texto = [word for word in word_tokens if word not in stopers]
        leters = " ".join(texto).lower()
        #print(leters.lower())
        lista_lista.append(leters)
    return lista_lista

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [16]:
stops = set(stopwords.words('english'))
listik = []
textlist_train = preprocesar(X_train, listik, stops, True)
text_clf= Pipeline([('tfidf', TfidfVectorizer()), ('ada_clf', AdaBoostClassifier(n_estimators=100))])
text_clf.fit(textlist_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [18]:
model_details = client.repository.store_model(text_clf, meta_props=model_properties, training_data=None)

#published_model = client.repository.store_model(model=pipeline, meta_props=model_props, training_data=X_train, training_target=y_train)

In [19]:
published_model_uid = client.repository.get_model_uid(model_details)
model_details = client.repository.get_details(published_model_uid)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [20]:
models_details = client.repository.list_models()

------------------------------------  --------  ------------------------  -----------------
GUID                                  NAME      CREATED                   FRAMEWORK
b9e260ef-e9c1-475a-ae3d-057d5fd20704  CritiKon  2020-04-15T02:27:23.317Z  scikit-learn-0.19
f226ecb5-154d-4b0d-a2d6-58bd3f5b8aeb  CritiKon  2020-04-15T02:27:06.509Z  scikit-learn-0.19
e5f02a3f-785d-4112-900e-b6096cfb4311  CritiKon  2020-04-15T02:12:53.504Z  scikit-learn-0.19
------------------------------------  --------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [21]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [22]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.77
             precision    recall  f1-score   support

          0       0.78      0.76      0.77       200
          1       0.76      0.78      0.77       200

avg / total       0.77      0.77      0.77       400

